In [ ]:
home_dir = homedir()

using Pkg; Pkg.activate(home_dir * ".julia/environments/DataFrames")

import DataFrames as DF
using CSV, Gadfly

In [ ]:
years = ["2017", "2018", "2019", "2020", "2021"]

folder_weather = home_dir * "Documents/github/BikeShareToronto/dataset/data/weather/"
files_weather = readdir(folder_weather)

for year in years
    files_weather_selected = []
    for file in files_weather
        if occursin(year, file)
            push!(files_weather_selected, file)
        end
    end

    df_all = DF.DataFrame()
    for file in files_weather_selected
        append!(df_all, DF.DataFrame(CSV.File(folder_weather * file)); cols = :union)
    end

    DF.describe(df_all, :all)
    df_weather = df_all[:, ["Temp (°C)", "Month"]]
    DF.rename!(df_weather, "Temp (°C)" => :temp, "Month" => :month)
    DF.dropmissing!(df_weather)

    df_weather[!,:month] = string.(df_weather[!,:month])
    xticks = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    for i in 1:12
        replace!(df_weather.month, string(i) => xticks[i])
    end

    set_default_plot_size(16cm, 9cm)

    p = plot(df_weather,
        x = :month,
        y = :temp,
        color = :month,
        Geom.boxplot,
        Guide.xlabel("Months"),
        Guide.ylabel("Temperature (°C)"),
        Guide.title("Monthly Weather in Toronto ($year)"),
        Guide.xticks(ticks=[0:13;]),
        Guide.colorkey(title = "Months"),
        Scale.y_continuous(format = :plain),
        Theme(background_color = "white")
    )

    img = SVG(home_dir * "Documents/github/BikeShareToronto/project/weather_$(year)_julia.svg", 16cm, 9cm)
    draw(img, p)
end